In [1]:
from openai import OpenAI
import os
import json
OpenAI.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
def generate_leetcode_question(difficulty: str, data_structures: list, programming_language: str):
    data_structures_str = ", ".join(data_structures)
    client = OpenAI()
    json_schema = {
        "challenge": "",
        "entry_function": "",
        "test_cases": [],
        "solution": ""
    }
    json_schema_str = json.dumps(json_schema, indent=2)
    prompt = f"""As an intelligent Leetcode Builder API, given the following criteria:
- Difficulty: {difficulty}
- Data Structures: {data_structures_str}
- Programming Language: {programming_language}

Create a leetcode question, make it a random scenario where the solution should involve the data structure for the best solution.

The response as follows:
1: "challenge": "text with the Leetcode challenge, give test cases and example result of test case and explain why"
2: "entry_function": "example in Python, e.g., def solve(arr: list) -> int: return -1"
3: "test_cases": "JSON format with input values and output values"
4: "solution": "one way to solve the challenge"

Your task is to generate a response that conforms to the following JSON schema:
{json_schema_str}

Please fill in the schema with appropriate values based on the criteria. Ensure the response strictly adheres to the given schema format."""

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": prompt},
        ],
        temperature=0,
    )
    return response
# Example usage
difficulty = "Medium"
data_structures = ["Array"]
programming_language = "python"
generated_text = generate_leetcode_question(difficulty, data_structures, programming_language)
leetcode = generated_text.copy()

In [3]:
leetcode_dict = json.loads(leetcode.choices[0].message.dict()['content'])
print(leetcode_dict)
print("--------------------------------")
print(leetcode_dict['challenge'])
print(leetcode_dict['entry_function'])
print(leetcode_dict['test_cases'])
print(leetcode_dict['solution'])

{'challenge': 'Given an array of integers, find the length of the longest subarray where the absolute difference between any two elements is less than or equal to 1. If there are multiple subarrays with the same length, return the length of any one of them. For example, given the array [1, 2, 2, 3, 1], the longest subarray with absolute difference less than or equal to 1 is [2, 2, 3] or [1, 2, 2], both with length 3.', 'entry_function': 'def longest_subarray(arr: list) -> int: return -1', 'test_cases': [{'input': [1, 2, 2, 3, 1], 'output': 3}, {'input': [1, 1, 1, 1, 1], 'output': 5}, {'input': [1, 2, 3, 4, 5], 'output': 2}, {'input': [1, 3, 5, 7, 9], 'output': 1}], 'solution': 'def longest_subarray(arr: list) -> int: \n    if not arr: return 0\n    max_len = 1\n    curr_len = 1\n    for i in range(1, len(arr)):\n        if abs(arr[i] - arr[i-1]) <= 1:\n            curr_len += 1\n            max_len = max(max_len, curr_len)\n        else:\n            curr_len = 1\n    return max_len'}


In [4]:
import threading    
def compile_and_run_function(code, test_cases):
    local_vars = {}
    exec(code, globals(), local_vars)
    function_name = list(local_vars.keys())[0]
    function = local_vars[function_name]
    def run_test_cases():
        for i, test in enumerate(test_cases):
            input_val = test["input"]
            expected_output = test["output"]
            params = []
            if (type(input_val) == dict):
                input_val_keys = input_val.keys()
                for key in input_val.keys():
                    params.append(input_val[key])
            else:
                params.append(input_val)
            try:
                actual_output = function(*params)
                assert actual_output == expected_output, f"Test #{i+1} failed: {input_val} -> {actual_output} != {expected_output}"
                print(f"Test #{i+1} passed: {input_val} -> {actual_output} == {expected_output}")
            except AssertionError as e:
                print(e)
            except Exception as e:
                print(f"Test #{i+1} raised an unexpected error: {e}")
    test_thread = threading.Thread(target=run_test_cases)
    test_thread.start()
    test_thread.join()
    
entry_function_code = leetcode_dict["entry_function"]
test_cases = leetcode_dict["test_cases"]
compile_and_run_function(entry_function_code, test_cases)

Test #1 failed: [1, 2, 2, 3, 1] -> -1 != 3
Test #2 failed: [1, 1, 1, 1, 1] -> -1 != 5
Test #3 failed: [1, 2, 3, 4, 5] -> -1 != 2
Test #4 failed: [1, 3, 5, 7, 9] -> -1 != 1
